# Conceptos aprendidos

## Librerías

In [ ]:
from functools import partial
from operator import itemgetter
from typing import Sequence

from dotenv import load_dotenv
from langchain.base_language import BaseLanguageModel
from langchain.chat_models import ChatOpenAI
from langchain.document_transformers import LongContextReorder
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import SQLRecordManager, index
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import BaseRetriever, Document, StrOutputParser
from langchain.schema.messages import BaseMessageChunk
from langchain.schema.runnable import Runnable, RunnableMap
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from src.langchain_docs_loader import LangchainDocsLoader, num_tokens_from_string

load_dotenv()

## Procesamiento de datos

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=50,
    length_function=num_tokens_from_string,
)

In [ ]:
keyword_docs = LangchainDocsLoader(
    include_output_cells=True,
    include_links_in_header=True,
).load()

splitted_docs = text_splitter.split_documents(keyword_docs)

filtered_docs = [
    doc
    for doc in splitted_docs
    if doc.page_content not in ("```", "```text", "```python")
]

len(filtered_docs)

## Indexación

### Almacenaje de documento en Vectorstore

In [ ]:
record_manager = SQLRecordManager(
    db_url="sqlite:///:memory:",
    namespace="langchain",
)

record_manager.create_schema()

embeddings = OpenAIEmbeddings()

vectorstore = Chroma(collection_name="langchain", embedding_function=embeddings)

indexing_result = index(
    docs_source=filtered_docs,
    record_manager=record_manager,
    vector_store=vectorstore,
    batch_size=1000,
    cleanup="full",
    source_id_key="source",
)

In [ ]:
indexing_result

### Obtención de los documentos almacenados

Si nuestra base de documentos inicial contenía documentos duplicados, éstos se han eliminado en el proceso de indexación. Por lo tanto, el número de documentos almacenados en Vectorstore podría ser menor que el número de documentos de la base inicial.

Al obtener los documentos almacenados en Vectorstore podemos tener una copia fidedigna de la base de datos inicial, pero sin duplicados. Esta copia puede ser utlizada para crear un nuevo índice o inicializar un retriever.

In [ ]:
vector_keys = vectorstore.get(
    ids=record_manager.list_keys(), include=["documents", "metadatas"]
)

docs_in_vectorstore = [
    Document(page_content=page_content, metadata=metadata)
    for page_content, metadata in zip(
        vector_keys["documents"], vector_keys["metadatas"]
    )
]

## Inicialización de retrievers

In [ ]:
keyword_retriever = BM25Retriever.from_documents(docs_in_vectorstore)
keyword_retriever.k = 5

semantic_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,
        "fetch_k": 50,
        "lambda_mult": 0.3,
    },
)

retriever = EnsembleRetriever(
    retrievers=[keyword_retriever, semantic_retriever],
    weights=[0.3, 0.7],
)

## Creación de RAG

### Prompts

In [ ]:
CONDENSE_QUESTION_TEMPLATE = """\
Given the following conversation and a follow up question, rephrase the follow up \
question to be a standalone question.

Chat History:
====================
{chat_history}
====================

Follow Up Input: {question}
Standalone Question:"""

SYSTEM_ANSWER_QUESTION_TEMPLATE = """\
You are an expert programmer and problem-solver, tasked with answering any question \
about 'Langchain' with high quality answers and without making anything up.

Generate a comprehensive and informative answer of 80 words or less for the \
given question based solely on the provided search results (URL and content). You must \
only use information from the provided search results. Use an unbiased and \
journalistic tone. Combine search results together into a coherent answer. Do not \
repeat text. Cite search results using [${{number}}] notation. Only cite the most \
relevant results that answer the question accurately. Place these citations at the end \
of the sentence or paragraph that reference them - do not put them all at the end. If \
different results refer to different entities within the same name, write separate \
answers for each entity.

If there is nothing in the context relevant to the question at hand, just say "Hmm, \
I'm not sure.". Don't try to make up an answer. This is not a suggestion. This is a rule.

Anything between the following `context` html blocks is retrieved from a knowledge \
bank, not part of the conversation with the user.

<context>
    {context}
</context>

REMBEMBER: If there is no relevant information within the context, just say "Hmm, \
I'm not sure.". Don't try to make up an answer. This is not a suggestion. This is a rule. \
Anything between the preceding 'context' html blocks is retrieved from a knowledge bank, \
not part of the conversation with the user.

Take a deep breath and relax. You are an expert programmer and problem-solver. You can do this.
You can cite all the relevant information from the search results. Let's go!"""

### Creación de cadena de retrieval

In [ ]:
def create_retriever_chain(
    llm: BaseLanguageModel[BaseMessageChunk],
    retriever: BaseRetriever,
    use_chat_history: bool,
):
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(CONDENSE_QUESTION_TEMPLATE)
    if not use_chat_history:
        initial_chain = (itemgetter("question")) | retriever
        return initial_chain
    else:
        condense_question_chain = (
            {
                "question": itemgetter("question"),
                "chat_history": itemgetter("chat_history"),
            }
            | CONDENSE_QUESTION_PROMPT
            | llm
            | StrOutputParser()
        )
        conversation_chain = condense_question_chain | retriever
        return conversation_chain

### Truncado de documentos recuperados a un número de documentos

In [ ]:
def get_k_or_less_documents(documents: list[Document], k: int):
    if len(documents) <= k:
        return documents
    else:
        return documents[:k]

### Reordenado de documentos recuperados

In [ ]:
def reorder_documents(documents: list[Document]):
    reorder = LongContextReorder()
    return reorder.transform_documents(documents)

###  Formateo de documentos recuperados

In [ ]:
def format_docs(docs: Sequence[Document]) -> str:
    formatted_docs: list[str] = []
    for i, doc in enumerate(docs):
        doc_string = f"<doc id='{i}'>{doc.page_content}</doc>"
        formatted_docs.append(doc_string)
    return "\n".join(formatted_docs)

### Creación de cadena de respuesta

In [ ]:
def create_answer_chain(
    llm: BaseLanguageModel[BaseMessageChunk],
    retriever: BaseRetriever,
    use_chat_history: bool,
    k: int = 5,
) -> Runnable:
    retriever_chain = create_retriever_chain(llm, retriever, use_chat_history)

    _get_k_or_less_documents = partial(get_k_or_less_documents, k=k)

    context = RunnableMap(
        {
            "context": (
                retriever_chain
                | _get_k_or_less_documents
                | reorder_documents
                | format_docs
            ),
            "question": itemgetter("question"),
            "chat_history": itemgetter("chat_history"),
        }
    )

    prompt = ChatPromptTemplate.from_messages(
        messages=[
            ("system", SYSTEM_ANSWER_QUESTION_TEMPLATE),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{question}"),
        ]
    )

    response_synthesizer = prompt | llm | StrOutputParser()
    response_chain = context | response_synthesizer

    return response_chain

## Interacción con el usuario

### Inicialización del chatbot

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.0)

answer_chain = create_answer_chain(
    llm=llm, retriever=retriever, use_chat_history=False, k=6
)

### Ejemplo 1

In [ ]:
question = "How to use .stream method in my chain with code example?"

In [ ]:
print(
    answer_chain.invoke(  # type: ignore
        {
            "question": question,
            "chat_history": [],
        }
    )
)

In [ ]:
keyword_docs = keyword_retriever.get_relevant_documents(
    query=question,
)

for i, doc in enumerate(keyword_docs, start=1):
    print(f"{i}. {doc.metadata['source']}: {doc.metadata.get('description', '')}")

In [ ]:
semantic_docs = semantic_retriever.get_relevant_documents(
    query=question,
)

for i, doc in enumerate(semantic_docs, start=1):
    print(f"{i}. {doc.metadata['source']}: {doc.metadata.get('description', '')}")

In [ ]:
ensemble_docs = retriever.get_relevant_documents(
    query=question,
)

for i, doc in enumerate(ensemble_docs, start=1):
    print(f"{i}. {doc.metadata['source']}: {doc.metadata.get('description', '')}")

### Ejemplo 2

In [ ]:
question = "How to use .batch method in my chain with code example?"

In [ ]:
print(
    answer_chain.invoke(  # type: ignore
        {
            "question": question,
            "chat_history": [],
        }
    )
)

In [ ]:
keyword_docs = keyword_retriever.get_relevant_documents(
    query=question,
)

for i, doc in enumerate(keyword_docs, start=1):
    print(f"{i}. {doc.metadata['source']}: {doc.metadata.get('description', '')}")

In [ ]:
semantic_docs = semantic_retriever.get_relevant_documents(
    query=question,
)

for i, doc in enumerate(semantic_docs, start=1):
    print(f"{i}. {doc.metadata['source']}: {doc.metadata.get('description', '')}")

In [ ]:
ensemble_docs = retriever.get_relevant_documents(
    query=question,
)

for i, doc in enumerate(ensemble_docs, start=1):
    print(f"{i}. {doc.metadata['source']}: {doc.metadata.get('description', '')}")